In [15]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
# load dataset
df = pd.read_csv("Most-Recent-Cohorts-Field-of-Study.csv", low_memory=False)

In [ ]:
#explore dataset
print(df.shape)
print(df.columns.tolist())
df.head()

In [ ]:
# filter for Bachelor programs
bachelors_df = df[df['CREDDESC'].str.contains("Bachelor's Degree", na=False)]
bachelors_df.shape

In [ ]:
# select relevent fields
selected_cols = [
    'INSTNM', 'CIPDESC', 'CREDLEV', 'CREDDESC',
    'DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_1YR', 'EARN_MDN_4YR',
    'EARN_MDN_5YR']
data = bachelors_df[selected_cols].copy()
data.shape

In [ ]:
# Convert debt and income to numeric
for column in ['DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_MDN_5YR']:
    data[column] = pd.to_numeric(data[column], errors='coerce')

# Drop rows with missing values in the chosen columns
df_clean = data.dropna(subset=['DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_MDN_5YR'])

In [ ]:
# Features for clustering
features = ['DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_MDN_5YR']
X = df_clean[features]

In [ ]:
# standardize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Elbow method to find optimal k
inertia = []
K = range(1, 10)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

In [ ]:
# Plot elbow curve
plt.figure(figsize=(8, 4))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
# KMeans clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
df_clean['Cluster'] = kmeans.fit_predict(X_scaled)

In [ ]:
# Display cluster centers
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
cluster_df = pd.DataFrame(cluster_centers, columns=features)
print(cluster_df)

In [ ]:
cluster_labels = {
    0: 'Moderate-Debt High-Earnings',
    1: 'Low-Debt Low-Earnings',
    2: 'High-Debt Moderate-Earnings'}

df_clean['Cluster_Label'] = df_clean['Cluster'].map(cluster_labels)

In [ ]:
# Scatter plot with clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_clean,
    x='DEBT_ALL_PP_ANY_MDN',
    y='EARN_MDN_5YR',
    hue='Cluster_Label',
    palette='Set1',
    alpha=0.7
)
plt.title("College Clusters by Debt and 5-Year Earnings")
plt.xlabel("Median Debt (All Students)")
plt.ylabel("Median Earnings (5 Years After Graduation)")
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

In [ ]:
cluster_summary = df_clean.groupby('Cluster')[['DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_MDN_5YR']].mean().round(2)
cluster_summary['Count'] = df_clean['Cluster'].value_counts().sort_index()
print("Cluster Summary:\n", cluster_summary)

In [ ]:
# calculate ROI
df_clean['Earnings_to_Debt'] = df_clean['EARN_MDN_5YR'] / df_clean['DEBT_ALL_PP_ANY_MDN']
df_clean['Earnings_to_Debt'] = df_clean['Earnings_to_Debt'].replace([float('inf'), -float('inf')], None)

In [ ]:
# pull top 3 best ROI programs per cluster
top_programs = df_clean.sort_values(by='Earnings_to_Debt', ascending=False).groupby('Cluster').head(3)
print("Top 3 programs by ROI in each cluster:\n", top_programs[['INSTNM', 'CIPDESC', 'DEBT_ALL_PP_ANY_MDN', 'EARN_MDN_5YR', 'Earnings_to_Debt']])